### Step 1: Import Dependencies

*Make sure to follow the installation first in the readme*

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

from client.yolov7_client.yolov7_triton_client import YoloV7_Triton_Inference_Client
from client.detectron2_client.detectron2_triton_client import Detectron2_Triton_Client
from client.utils.exports import export_image_to_fo, export_video_to_fo

from ipywidgets import interact, Dropdown

import json
import imghdr
import sys

***

### Step 2: Start Triton Inference Server and CVAT 

(*will likely be combined into single start up script*)

2a. Triton Inference Server: run the following command, replace source with models' path

    docker run --gpus all --rm --ipc=host --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 \
    -p8000:8000 -p8001:8001 -p8002:8002 --mount type=bind,source="path/to/triton/models",destination=/models \
    nvcr.io/nvidia/tritonserver:22.06-py3 tritonserver --model-repository=/models --strict-model-config=false \
    --log-verbose 1

        The command is as follows:
        * --gpus all: specifies to use all available GPU on device
        * --ipc=host: docker will share resource with host machine
        * --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864: set up container runtime configs
        * -p8000:8000: expose port 8000 for HTTP 
        * -p8001:8001: expose port 8001 for GRPC 
        * -p8002:8002: expose port 8002 for metrics 
        * -mount type=bind,source="path/to/triton/models",destination=/models: mount models to container
        * nvcr.io/nvidia/tritonserver:22.06-py3: pull from triton server image
        * tritonserver --model-repository=/models --strict-model-config=false \
            --log-verbose 1: starts triton inference server
    

2b. CVAT: install local server as per instructions found [here](https://opencv.github.io/cvat/docs/administration/basics/installation/)

   Once installed, to start a local CVAT server at port 8080 run the following commands

    cd /path/to/cvat/clone/ \
    CVAT_VERSION=v2.3.0 docker compose up -d  

*note: cvat only works with fiftyone's api up to cvat v2.3.0*


***

### Step 3: Start Fiftyone Instance Locally

Run the following cell, and navigate to localhost:5151

*If the error "Could not connect session, trying again in 10 seconds" occurs, 
it is likely the session is already started.  In that case nagivate to 
localhost:5151 and see if a session is already started*

In [2]:
# start fiftyone

session = fo.launch_app(auto=False)
session.open_tab()


Could not connect session, trying again in 10 seconds

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

(Optional) 3a. Upload labeled dataset (if wish to evaluate model performance)

Demo: Upload COCO validation dataset to Fiftyone with labels

In [20]:
dataset_demo = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    dataset_name="baseline_demo"
)
dataset_demo.persistent = True

Found annotations at 'C:\Users\Alex Lin\fiftyone\coco-2017\raw\instances_val2017.json'
Found 100 (< 5000) downloaded images; must download full image zip
 100% |██████|    6.1Gb/6.1Gb [1.4m elapsed, 0s remaining, 86.4Mb/s]      
Extracting images to 'C:\Users\Alex Lin\fiftyone\coco-2017\validation\data'
Writing annotations to 'C:\Users\Alex Lin\fiftyone\coco-2017\validation\labels.json'
Dataset info written to 'C:\Users\Alex Lin\fiftyone\coco-2017\info.json'
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [29.7s elapsed, 0s remaining, 163.4 samples/s]      
Dataset 'baseline_demo' created


(Optional) 3b. Alternatively, any desired dataset with labels can be uploaded to Fiftyone below 
from a annotations.json file with the format by providing the file 

    Images
        {
            "path/to/image": {
                "detections": [
                    {
                        "bbox": [<top-left-x>, <top-left-y>, <width>, <height>],
                        "label": obj_class,
                        "confidence": 0.00-1.00
                    }
                ],
                "tags": [training/validation/testing]
            }
            ...
        }

                            OR

    Videos
        {
            "path/to/video": {
                1: {
                    "detections": [
                        {
                            "bbox": [<top-left-x>, <top-left-y>, <width>, <height>],
                            "label": obj_class,
                            "confidence": 0.00-1.00
                        }
                    ],
                    tags": [training/validation/testing],
                }
                2: { ... }
            }
            ...
        }

*note: vscode jupyter does not support fileupload widget, so uploads will be specified \
by the file path to the json file*

In [3]:
# path of file to be uploaded to fiftyone as labeled dataset
json_file = "C:\\Users\\Alex Lin\\Desktop\\baseline_system\\data\\inference\\images\\test1.json"
# name of the new dataset
dataset_name = 'inference_test'
# name of label of detections in fiftyone
label_field = 'ground_truth'   


with open(json_file) as json_file:
    try:
        # laod .json file
        annotations = json.load(json_file)
        # check for empty file
        if not annotations:
            print("Error: .json file is empty")
            sys.exit(1)
        # check if image or video annotations
        if imghdr.what(list(annotations.keys())[0]):
            print("Exporting image annotations...")
            export_image_to_fo(annotations=annotations, dataset=dataset_name, label_field=label_field)
        else:
            print("Exporting video annotations...")
            export_video_to_fo(annotations=annotations, dataset=dataset_name, label_field=label_field)
    except ValueError as e:
        print("Invalid .json file")
      

Exporting image annotations...
Dataset does not exist in fiftyone, creating new                 dataset by default.  Make sure the dataset set to persistent                 if using existing fiftyone datset
 100% |█████████████████████| 4/4 [97.7ms elapsed, 0s remaining, 40.9 samples/s]      


***

### Step 4: Initilize the Model Client, Run Inference


4a. initialize the variables associated with the model as well as the class labels 

*Note: currently all demo models are trained on coco_classes.  If a custom model implements \
separate class labels, please make sure to replace the classes with the correct class label \
for use in annotation later*

    - url: Inference server URL, default localhost:8001
    - model_info: Print model status, configuration and statistics
    - verbose: Enable verbose client output
    - client_timeout: Client timeout in seconds, default no timeout
    - ssl: Enable SSL encrypted channel to the server
    - root_certificates: File holding PEM-encoded root certificates, default none
    - private_key: File holding PEM-encoded private key, default is none
    - certificate_chain: File holding PEM-encoded certicate chain default is none
    - client_timeout: Client timeout in seconds, default no timeout
    - width: Inference model input width, default 640
    - height: Inference model input height, default 640

In [3]:
url_triton='localhost:8001'
model_info=False
verbose=False
client_timeout=None
ssl=False
root_certificates=None
private_key=None
certificate_chain=None
width=640
height=640

4b. select the desired client

*If new clients are loaded onto the triton inference server, simply append them \
to the client list and add the conditionals in init_client() to create a client. \
Make sure to import the client*

In [4]:
# initialize the client
client_list = ["select", "yolov7", "detectron2"]
clientW = Dropdown(options=client_list)
client = [None]

@interact(client_choice=clientW)
def init_client(client_choice):
    """
    Initializes client with choice from dropdown

    :params:
        - client_choice: chosen from client_list
    """
    if client_choice == "select":
        return
    elif client_choice == "yolov7":
        client[0] = (YoloV7_Triton_Inference_Client(
            url=url_triton,
            model_info=model_info,
            verbose=verbose,
            client_timeout=client_timeout,
            ssl=ssl,
            root_certificates=root_certificates,
            private_key=private_key,
            certificate_chain=certificate_chain,
            width=width,
            height=height
        ))
    elif client_choice == "detectron2":
        client[0] = (Detectron2_Triton_Client(
            url=url_triton,
            model_info=model_info,
            verbose=verbose,
            client_timeout=client_timeout,
            ssl=ssl,
            root_certificates=root_certificates,
            private_key=private_key,
            certificate_chain=certificate_chain,
            width=1344,
            height=1344
        ))

interactive(children=(Dropdown(description='client_choice', options=('select', 'yolov7', 'detectron2'), value=…

(Optional) 4c. Select the existing dataset on fiftyone to use

This would ideally be a labelled dataset to have the model make predictions on,
which would then be imported back to fiftyone/cvat for validation and to calculate
analysis metrics on.

In [5]:
datasets = ["select"] + fo.list_datasets()
datasetsW = Dropdown(options=datasets)
# create a global var, since dataset object don't seem permanent across cells
dataset = [None]

@interact(dataset_choice=datasetsW)
def init_dataset(dataset_choice):
    if dataset_choice == "select":
        return
    # load dataset 
    dataset[0] = (fo.load_dataset(dataset_choice))

interactive(children=(Dropdown(description='dataset_choice', options=('select', 'inference_test', 'inference_t…

4d. Set Runtime Configs 

*Make sure these configs are for your desired inference mode to avoid errors*

For images 
   - input_: Input directory to load from in image 
       NOTE: directory must only contain image files
   - vis: Show visualization of prediction on computer, default false
   - output_: Output directory, default no output saved
   - fo_dataset: fiftyone dataset of labeled images to validate/test model with
       default None, no dataset used
   - create_fo_ds: name of new fiftyone dataset to create, **ONLY** used in combination \
       with input_ (inferencing locally, want to visualize results only)
   - json_out: Output directory for annotations outputs, includes filename
       e.g. /my/output/file/path/myfile.json
   - tags: list of tags to organize inference results in fiftyone

For videos
   - input_: Input directory to load from in video OR a fiftyone dataset \
       of labeled images to validate/test model with 
       NOTE: directory must only contain video files
   - vis: Show visualization of prediction on computer, default false
   - fo_dataset: Dataset name to export predictions to fiftyone, 
       default '', no export
   - create_fo_ds: name of new fiftyone dataset to create, **ONLY** used in combination \
       with input_ (inferencing locally, want to visualize results only)
   - json_out: Output directory for annotations outputs, includes filename
       e.g. /my/output/file/path/myfile.json            
   - output_: Output directory, default no output saved
   - fps: Video output fps, default 24.0 FPS
   - tags: list of tags to organize inference results in fiftyone

Dummy requires no input 

**Can either have input form local directory or from fiftyone, not both**

*Operations to export to fiftyone and export locally can both be down.  Local \
visualization options (not listed here) is limited and should only be used for \
development*

In [6]:
# set this to dataset[0]
fo_dataset = dataset[0]

# OR

# set these fields manually
input_ = ''
output_ = ''
create_fo_ds = ''
json_out = ''
fps = 24.0

# tags to be added
tags = ["validation"]

4e. Run Inference in Desired Mode

**If exported to fiftyone, navigate to your fiftyone client (should be a webpage \
at localhost:5151), refresh, and select the dataset exported to to see inference \
results**

In [7]:
inference_choice = ['select', 'image', 'video', 'dummy']
inferenceW = Dropdown(options=inference_choice)

@interact(mode=inferenceW)
def inference(mode):
    """
    Runs inference through Triton Inference Server 

    :params:
        - mode, media type to run through server, chosen from inference_choice
    """
    if mode == 'select':
        return
    elif mode == 'image':
        client[0].infer_image(
            input_=input_, 
            output_=output_, 
            fo_dataset=fo_dataset, 
            create_fo_ds=create_fo_ds,
            json_out=json_out, 
            tags=tags
            )
    elif mode == 'video':
        client[0].infer_video(
            input_=input_, 
            output_=output_, 
            fo_dataset=fo_dataset, 
            create_fo_ds=create_fo_ds,
            fps=fps, 
            json_out=json_out, 
            tags=tags
            )
    elif mode == 'dummy':
        client[0].infer_dummy()

interactive(children=(Dropdown(description='mode', options=('select', 'image', 'video', 'dummy'), value='selec…

***

### Step 5: Validation Via CVAT

Once inference has completed, validation may be performed through CVAT.  The dataset 
visualized in fiftyone would then be uploaded to CVAT, ground truth can be annotated,
and the result imported back to fiftyone for analysis.


5a. Config for Validation

In [49]:
COLORS = ["n/a", "red", "blue", "green", "yellow", "purple", "pink", "orange", "brown", "black", "white", "gray", 
          "gold", "silver", "navy blue", "sky blue", "lime green", "teal", "indigo", "magenta", "violet", 
          "khaki", "salmon", "crimson", "lavender", "plum", "blue violet", "olive", "cyan", "maroon", "beige"]

# change this to the class labels your model of choice was trained on.  
# the default demo detectron2 and yolov7 labels are trained on COCO labels
COCO_CLASSES=["person","bicycle","car","motorcycle","airplane","bus","train","truck","boat","traffic light","fire hydrant", 
              "stop sign","parking meter","bench","bird","cat","dog","horse","sheep","cow","elephant","bear","zebra",
              "giraffe","backpack","umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite",
              "baseball bat","baseball glove","skateboard","surfboard","tennis racket","bottle","wine glass","cup","fork",
              "knife","spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hot dog","pizza","donut",
              "cake","chair","couch","potted plant","bed","dining table","toilet","tv","laptop","mouse","remote","keyboard",
              "cell phone","microwave","oven","toaster","sink","refrigerator","book","clock","vase","scissors","teddy bear",
              "hair drier", "toothbrush"]

label_schema = {
    "novel_detections": {
        "type": "detections",
        "classes": ["novel_object"],
        "attributes": {
            "novelty": {
                "type": "select",
                "values": ["not seen before", "new presentation", "idfk"],
                "default": "not seen before",
            },
        },
    },
    "model_detections": {},
}

anno_key = "test_run_images36"
launch_editor=False
url_cvat="http://localhost:8080"
username="django"
password="bfc"

5b. Run CVAT Validation

Specific Metrics to Pick Data to Validate, **come up with them**


In [50]:
# create specific view for low confidence model predictions
low_conf_view = (
    dataset[0] \
    .filter_labels("model_detections", F("confidence") < 0.6)
    .sort_by(F("predictions.detections").length(), reverse=True)
    ) \

# fastdup, cleanlab

sample_id = low_conf_view.head(3)
view = dataset[0].select(sample_id)

anno_keys = dataset[0].list_annotation_runs()

# check if anno key already exists
if anno_key in anno_keys:
    # Delete tasks from CVAT
    results = dataset[0].load_annotation_results(anno_key)
    if results is not None:
        results.cleanup()
    dataset[0].delete_annotation_run(anno_key)

# send samples to CVAT
view.annotate(
    anno_key,
    label_schema=label_schema,
    launch_editor=True,
    allow_additions=True,
    allow_deletions=False,
    allow_label_edits=True,
    allow_index_edits=True,
    allow_spatial_edits=True,
    url="http://localhost:8080",
    username="django",
    password="arclight"
)

Found existing field 'model_detections' with multiple types ['detections', 'instances']. Only the 'detections' will be annotated
Uploading samples to CVAT...
Launching editor at 'http://localhost:8080/tasks/3/jobs/3'...


5c. Merge Dataset Back to Fiftyone and Cleanup CVAT

In [51]:
# merge annotations back to Fiftyone dataset
dataset[0].load_annotations(anno_key)
dataset[0].load_annotation_view(anno_key)

# Delete tasks from CVAT
results = dataset[0].load_annotation_results(anno_key)
results.cleanup()

# Delete run record (not the labels) from FiftyOne
dataset[0].delete_annotation_run(anno_key)

Download complete
Loading labels for field 'novel_detections'...
 100% |█████████████████████| 1/1 [54.6ms elapsed, 0s remaining, 18.3 samples/s] 
Loading labels for field 'model_detections'...
                                                                              100% |█████████████████████| 2/2 [54.7ms elapsed, 0s remaining, 36.6 samples/s] 
Deleting tasks...
 100% |█████████████████████| 1/1 [320.4ms elapsed, 0s remaining, 3.1 samples/s] 


***

### Step 6. Analysis and Metrics

***

### (Optional) Step 7. Cleanup

Make sure to select the proper dataset by running the cell in step 5c.

**WARNING: The follow will delete the selected samples from a dataset in fiftyone, \
only run if the selected samples in the dataset is to be deleted as they cannot \
be recovered**

*if error 'name 'session' is not defined' is thrown, restart kernel and rerun \
everything*

In [8]:
# delete selected samples
delete_view = dataset[0].select(session.selected)
dataset[0].delete_samples(delete_view)

**WARNING: The follow will delete the selected dataset from fiftyone, \
only run if the selected dataset in the dataset is to be deleted as they cannot \
be recovered**

*Fiftyone might have to be restarted (close tab and rerun step 4) if the \
deleted dataset is currently being viewed in the webapp*

In [6]:
dataset[0].delete()

***